In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

%load_ext autoreload
%autoreload 2

from collections import defaultdict

from src.utilities.gridSearch import GridSearchEstimators

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from sklearn.model_selection       import train_test_split, GridSearchCV
from sklearn.utils                 import resample

from sklearn.ensemble              import RandomForestClassifier, RandomForestRegressor
from sklearn.svm                   import SVC
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.linear_model          import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes           import GaussianNB
from sklearn.ensemble              import AdaBoostClassifier, GradientBoostingClassifier

from sklearn.metrics               import classification_report, f1_score, precision_score, recall_score
from sklearn.metrics               import make_scorer

# Presentation Notebook 1

This notebook is here to summarize our findings for both the feature-engineered ML model and the sparse grades matrix ML model.

The students part of this effort are: Skyler Shi, Danny Ferriss, Mit Kotak, Jarett Brunner, Divya Bhati. \
The instructor of our efforts is: Hannah Christenson.

# Feature-Engineered ML Model

Our initial idea was to engineer several features by intuition and use those features to predict the grades of GIES students. \
These are the features we finally decided to assemble:
- Previous Semester GPA
- Cumulative GPA to date
- Number of Credits to date
- Concentration Cumulative GPA
- Concentration Number of Credits
- Business Core Cumulative GPA
- Business Core Number of Credits
- Prerequisites GPA

In [4]:
ml_df = pd.read_csv('src/data/ml_ready.csv').drop(columns = ['Unnamed: 0'])
ml_df.head()

,EDW_PERS_ID,term_count,Majors_List,Course_name,course_grade,prev_sem_GPA,cum_GPA,num_credits,cum_GPA_concentration,num_credits_concentration,cum_GPA_business_core,num_credits_business_core,prereq_GPA
0,25539,1,Accountancy,BADM 449,B+,3.50,0.00,0.0,0.0,0.0,0.00,0.0,0.0
1,25539,2,Accountancy,ACCY 301,A,3.33,3.33,3.0,0.0,0.0,3.33,3.0,0.0
2,25539,3,Accountancy,ACCY 303,A,4.00,3.67,6.0,4.0,3.0,3.33,3.0,0.0
3,25539,4,Accountancy,ACCY 302,A,4.00,3.78,9.0,4.0,6.0,3.33,3.0,0.0
4,25539,5,Accountancy,ACCY 304,B+,4.00,3.83,12.0,4.0,9.0,3.33,3.0,0.0


## 0. Before we did any modeling, we cleaned up the distribution of grades

We combined honors grades with non-honors grades. For example **"AH"** is combined with **"A"**. We thought this is justifiable because for most classes, most of the coursework of honors students and non-honors students is still the same.

We also dropped grades with rare occurences such as **['W', 'PS', 'ABS', 'AU', 'NR']** because there were too few samples for them to be effectively predicted and they described occurences where the student doesn't finish the class. Whether a student finishes a class is not under our scope of investigation.

In [5]:
def combine_honors_grades(grade):
    if grade[-1] == 'H':
        return grade[0:-1]
    else:
        return grade

ml_df.course_grade = ml_df.course_grade.apply(combine_honors_grades)
ml_df = ml_df[~ml_df.course_grade.isin(['W', 'PS', 'ABS', 'AU', 'NR'])]

In [6]:
features = ['prev_sem_GPA'              ,
            'cum_GPA'                    ,
            'num_credits'                ,
            'cum_GPA_concentration'      ,
            'num_credits_concentration'  ,
            'cum_GPA_business_core'      ,
            'num_credits_business_core'  ,
            'prereq_GPA'                 ]

X = ml_df[features]
y = ml_df['course_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

## 1. First, we tried predicting the letter grades.

There were simply too many classes and not enough samples with bad grades for the machine learning model to be accurate.

Below is an example of predictions being run on BADM 300.

In [7]:
# get only samples that are BADM 300
badm300 = ml_df[ml_df.Course_name == "BADM 300"]

# train test split
X = badm300[features]
y = badm300['course_grade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# fit random forest classifier model
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
predictions = rfc.predict(X_test)
print(classification_report(y_test, predictions, zero_division = 0))

              precision    recall  f1-score   support

           A       0.32      0.56      0.41       185
          A+       0.30      0.26      0.28       117
          A-       0.07      0.05      0.06       103
           B       0.19      0.21      0.20       110
          B+       0.16      0.13      0.14        67
          B-       0.08      0.02      0.03        52
           C       0.10      0.06      0.08        33
          C+       0.07      0.03      0.04        35
          C-       0.00      0.00      0.00        17
           D       0.00      0.00      0.00         8
          D+       0.00      0.00      0.00         4
          D-       0.00      0.00      0.00         1
           F       0.00      0.00      0.00         4

    accuracy                           0.24       736
   macro avg       0.10      0.10      0.09       736
weighted avg       0.19      0.24      0.20       736



## 2. Seeing the results, we decided to predict the Grades Bucketed.

We decided to bucket together grades to promote separability within our models.
- **['A+', 'A', 'A-', 'B+', 'B', 'B-']** are deemed **"good"** grades
- **['C+', 'C', 'C-', 'D+', 'D', 'D-', 'F']** are deemed **"bad"** grades

In [8]:
def bucket_grades(x):
    if x in ['A+', 'A', 'A-', 'B+', 'B', 'B-']:
        return "good"
    if x in ['C+', 'C', 'C-', 'D+', 'D', 'D-', 'F']:
        return "bad"

grades_bucketed_df = ml_df.copy(deep = True)
grades_bucketed_df['course_grade'] = grades_bucketed_df['course_grade'].apply(bucket_grades)

In [9]:
grades_bucketed_df.course_grade.value_counts()

good    121342
bad      12053
Name: course_grade, dtype: int64

In [10]:
# get only samples that are BADM 300
badm300 = grades_bucketed_df[grades_bucketed_df.Course_name == "BADM 300"]

# train test split
X = badm300[features]
y = badm300['course_grade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

With our grades bucketed, we only have to predict two classes - "good grade" and "bad grade". The machine learning model should not choke as much. \
We run a gridsearch on some popular classifiers and track their performance:

In [11]:
models = {
    'RandomForest': RandomForestClassifier(),
    'KNN'         : KNeighborsClassifier(weights = "distance"),
    'SVM'         : SVC(class_weight = "balanced"),
    'NaiveBayes'  : GaussianNB(),
    'AdaBoost'    : AdaBoostClassifier(),
    'GradBoost'   : GradientBoostingClassifier()
    
}

params = {
    'RandomForest': {
    },
    'KNN': {
    },
    'SVM': {   
    },
    'NaiveBayes': {
    },
    'AdaBoost': {
    },
    'GradBoost': {
    }
}

scoring = {
    'precision (good)': make_scorer(precision_score, pos_label = 'good'),
    'precision (bad)':  make_scorer(precision_score, pos_label = 'bad'),
    'recall (good)':    make_scorer(recall_score, pos_label = 'good'),
    'recall (bad)':     make_scorer(recall_score, pos_label = 'bad'),
    'f1 (good)':        make_scorer(f1_score, pos_label = 'good'),
    'f1 (bad)':         make_scorer(f1_score, pos_label = 'bad'),
}
gs = GridSearchEstimators(models = models, params = params, scoring = scoring)
gs.fit(X_train, y_train)
gs.report()

Running GridSearchCV for RandomForest
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    1.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Running GridSearchCV for KNN
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Running GridSearchCV for SVM
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Running GridSearchCV for NaiveBayes
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Running GridSearchCV for AdaBoost
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.5s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Running GridSearchCV for GradBoost
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.6s finished


,Estimator,Parameters,mean_precision (good),mean_precision (bad),mean_recall (good),mean_recall (bad),mean_f1 (good),mean_f1 (bad)
0,RandomForest,{},0.885100,0.492100,0.971800,0.176500,0.926400,0.259400
1,KNN,{},0.868100,0.147500,0.971400,0.035900,0.916800,0.057500
2,SVM,{},0.919900,0.241100,0.710900,0.595900,0.801700,0.342800
3,NaiveBayes,{},0.900300,0.380000,0.915800,0.337700,0.908000,0.357400
4,AdaBoost,{},0.887200,0.518300,0.973000,0.192000,0.928100,0.276500
5,GradBoost,{},0.886300,0.525900,0.974200,0.184200,0.928200,0.271400


Even with grades bucketed into two classes, we observe that for all ML models tried, the metrics for the good grade class are all very high, but the metrics for the bad grade class are all very low. The focus of our ML model should be to accurately predict the bad grade class, as that represents the students at risk.

Hence, we definitely need to improve precision, recall, f1 for the bad grade class.

## 3. We Correct Class Imbalance to Improve Bad Grade Class metrics

Without any sample size correction, ML models are all biased towards predicting good grade class more accurately because that produces the highest overall accuracy. \
We increase the number of bad grade samples to be the same as good grade samples through up-sampling.

In [12]:
badm300 = grades_bucketed_df[grades_bucketed_df.Course_name == "ECON 302"]

In [13]:
# Separate majority and minority classes
grades_good = badm300[badm300.course_grade == "good"]
grades_bad  = badm300[badm300.course_grade == "bad"]
 
# Upsample minority class
badm300_bad_upsampled = resample(grades_bad, 
                                 replace = True,     # sample with replacement
                                 n_samples = badm300.course_grade.value_counts()['good'],    # to match majority class
                                 random_state = 123) # reproducible results
 
# Combine majority class with upsampled minority class
badm300_upsampled = pd.concat([grades_good, badm300_bad_upsampled])
 
# Display new class counts
badm300_upsampled.course_grade.value_counts()

bad     2725
good    2725
Name: course_grade, dtype: int64

In [14]:
# train test split
X = badm300_upsampled[features]
y = badm300_upsampled['course_grade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [15]:
models = {
    'RandomForest': RandomForestClassifier(),
    'KNN'         : KNeighborsClassifier(weights = "distance"),
    'SVM'         : SVC(class_weight = "balanced"),
    'NaiveBayes'  : GaussianNB(),
    'AdaBoost'    : AdaBoostClassifier(),
    'GradBoost'   : GradientBoostingClassifier()
    
}

params = {
    'RandomForest': {
    },
    'KNN': {
    },
    'SVM': {   
    },
    'NaiveBayes': {
    },
    'AdaBoost': {
    },
    'GradBoost': {
    }
}

scoring = {
    'precision (good)': make_scorer(precision_score, pos_label = 'good'),
    'precision (bad)':  make_scorer(precision_score, pos_label = 'bad'),
    'recall (good)':    make_scorer(recall_score, pos_label = 'good'),
    'recall (bad)':     make_scorer(recall_score, pos_label = 'bad'),
    'f1 (good)':        make_scorer(f1_score, pos_label = 'good'),
    'f1 (bad)':         make_scorer(f1_score, pos_label = 'bad'),
}
gs = GridSearchEstimators(models = models, params = params, scoring = scoring)
gs.fit(X_train, y_train)
gs.report()

Running GridSearchCV for RandomForest
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Running GridSearchCV for KNN
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Running GridSearchCV for SVM
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Running GridSearchCV for NaiveBayes
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Running GridSearchCV for AdaBoost
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.6s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Running GridSearchCV for GradBoost
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=3)]: Done   5 out of   5 | elapsed:    0.8s finished


,Estimator,Parameters,mean_precision (good),mean_precision (bad),mean_recall (good),mean_recall (bad),mean_f1 (good),mean_f1 (bad)
0,RandomForest,{},0.875600,0.833800,0.821700,0.884000,0.847500,0.858000
1,KNN,{},0.867100,0.732500,0.666400,0.896300,0.750900,0.805300
2,SVM,{},0.671900,0.723100,0.754800,0.634200,0.710800,0.675600
3,NaiveBayes,{},0.686900,0.724400,0.745200,0.663000,0.714700,0.692100
4,AdaBoost,{},0.765400,0.737300,0.719400,0.780400,0.741300,0.757900
5,GradBoost,{},0.781500,0.751300,0.734100,0.795400,0.756600,0.772400


These precision, recall and f1 numbers are much better for the bad grade class after correcting for imbalance. \
We are very happy with these results.

## 4. More Tuning

The RandomForest model seems promising. We tune it some more.

After trying more hyperparameters, we don't notice any significant improvements. Using default hyperparameters of Random Forest Classifier is just fine.

In [17]:
models = {
    'RandomForest': RandomForestClassifier(criterion = 'entropy')
}

params = {
    'RandomForest': {
        'max_depth': [None, 5, 10, 20],
        'min_samples_split': [2, 5, 20],
        'n_estimators': [100, 50, 200]
    }
}

scoring = {
    'precision (good)': make_scorer(precision_score, pos_label = 'good'),
    'precision (bad)':  make_scorer(precision_score, pos_label = 'bad'),
    'recall (good)':    make_scorer(recall_score, pos_label = 'good'),
    'recall (bad)':     make_scorer(recall_score, pos_label = 'bad'),
    'f1 (good)':        make_scorer(f1_score, pos_label = 'good'),
    'f1 (bad)':         make_scorer(f1_score, pos_label = 'bad'),
}
gs = GridSearchEstimators(models = models, params = params, scoring = scoring)
gs.fit(X_train, y_train)
gs.report()

Running GridSearchCV for RandomForest
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.1s
[Parallel(n_jobs=3)]: Done 180 out of 180 | elapsed:   30.4s finished


,Estimator,Parameters,mean_precision (good),mean_precision (bad),mean_recall (good),mean_recall (bad),mean_f1 (good),mean_f1 (bad)
0,RandomForest,"{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}",0.870000,0.827600,0.814700,0.879000,0.841200,0.852400
1,RandomForest,"{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}",0.874400,0.828000,0.814300,0.884000,0.843100,0.855000
2,RandomForest,"{'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}",0.875600,0.833600,0.821200,0.884000,0.847200,0.857800
3,RandomForest,"{'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100}",0.864700,0.832200,0.822100,0.872100,0.842600,0.851500
4,RandomForest,"{'max_depth': None, 'min_samples_split': 5, 'n_estimators': 50}",0.867500,0.830300,0.818900,0.875800,0.842300,0.852300
5,RandomForest,"{'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200}",0.868900,0.831300,0.819800,0.877200,0.843400,0.853400
6,RandomForest,"{'max_depth': None, 'min_samples_split': 20, 'n_estimators': 100}",0.830100,0.797000,0.783900,0.839700,0.805900,0.817400
7,RandomForest,"{'max_depth': None, 'min_samples_split': 20, 'n_estimators': 50}",0.833300,0.797100,0.782900,0.843800,0.807000,0.819500
8,RandomForest,"{'max_depth': None, 'min_samples_split': 20, 'n_estimators': 200}",0.834800,0.801500,0.788500,0.844300,0.810500,0.822000
9,RandomForest,"{'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}",0.780200,0.734600,0.707400,0.800500,0.741100,0.765400


## 5. Now Let's see how RandomForest Classifier performs on multiple core courses.

The courses examined are either courses that were taken the most throughout 2009 - 2018 or courses that students scored the lowest in. We can easily examine any set of courses GIES wants us to examine too.

As we can see the Random Forest model works well on a variety of courses! For all classes, the Random Forest model is able to predict grades with an accuracy of around 85%.

In [18]:
courses = ['BADM 300', 'ECON 302', 'BADM 449', 'ECON 203', 'BADM 320',
           'BADM 310', 'FIN 221', 'ACCY 201', 'ACCY 202', 'BUS 388',
           'ECON 202', 'FIN 300', ]

In [37]:
a = {
    'a':  123,
    'b': 123
}
b = list(a.keys())
b

['a', 'b']

In [45]:
results = defaultdict(list)

for course in courses:
    course_df = grades_bucketed_df[grades_bucketed_df.Course_name == course]
    
    # Separate majority and minority classes
    grades_good = course_df[course_df.course_grade == "good"]
    grades_bad  = course_df[course_df.course_grade == "bad"]

    # Upsample minority class
    course_bad_upsampled = resample(grades_bad, 
                                     replace = True,     # sample with replacement
                                     n_samples = course_df.course_grade.value_counts()['good'],    # to match majority class
                                     random_state = 123) # reproducible results

    # Combine majority class with upsampled minority class
    course_upsampled = pd.concat([grades_good, course_bad_upsampled])
    
    # train test split
    X = course_upsampled[features]
    y = course_upsampled['course_grade']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    
    
    # train course RFC model
    rfc = RandomForestClassifier(criterion = 'entropy', random_state = 42)
    
    rfc.fit(X_train, y_train)
    pred = rfc.predict(X_test)
    class_results = classification_report(y_test, pred, output_dict = True)
    results['Course'].append(course)
    for label, d in class_results.items():
        if label in ['macro avg', 'weighted avg']:
            continue
        if label == 'accuracy':
            results['accuracy'].append(class_results['accuracy'])
            continue
        for metric, val in d.items():
            results[(label, metric)].append(val)

multiIdxCols = [v for v in results.keys() if v not in ['Course', 'accuracy']]
cols = pd.MultiIndex.from_tuples(multiIdxCols)
res = pd.DataFrame(results, columns = cols, index = results['Course'])
res

bad                                  good                      \
         precision    recall  f1-score support precision    recall  f1-score   
BADM 300  0.897582  1.000000  0.946027     631  1.000000  0.888199  0.940789   
ECON 302  0.851852  0.902804  0.876588     535  0.900574  0.848649  0.873840   
BADM 449  0.989943  1.000000  0.994946     689  1.000000  0.990210  0.995081   
ECON 203  0.811133  0.879310  0.843847     464  0.882353  0.815534  0.847629   
BADM 320  0.998523  0.969871  0.983988     697  0.971888  0.998624  0.985075   
BADM 310  0.958730  0.922137  0.940078     655  0.928771  0.962373  0.945274   
FIN 221   0.850258  0.984064  0.912281     502  0.982379  0.836773  0.903749   
ACCY 201  0.911392  0.808989  0.857143     534  0.839370  0.926957  0.880992   
ACCY 202  0.931699  1.000000  0.964642     532  1.000000  0.934122  0.965939   
BUS 388   0.945554  1.000000  0.972015     521  1.000000  0.946809  0.972678   
ECON 202  0.855114  0.809140  0.831492     372  0.828087  0.870229  0.848635   
FIN 300   0.808824  0.882353  0.843990     187  0.876404  0.800000  0.836461   

                  
         support  
BADM 300     644  
ECON 302     555  
BADM 449     715  
ECON 203     515  
BADM 320     727  
BADM 310     691  
FIN 221      533  
ACCY 201     575  
ACCY 202     592  
BUS 388      564  
ECON 202     393  
FIN 300      195

# Sparse Matrix Classification Model

Another approach we attempted was the sparse matrix classification model. In this approach, there are thousands of features. Each feature represents a course and is numeric grade of the student who took the course. If the student has not taken the course, the numeric grade of the student is. 0.0 (hence the sparseness of the matrix).

We assume that having taken courses gives the student knowledge that contributes to their success in taking a specific course. Their grade in that course is a proxy for their mastery of the material. The higher the grade, the better their mastery.

The code for assembling the sparse matrices is computationally intensive and therefore omitted in this notebook. We directly read in the sparse matrices from pickle files. See`assemble_ml_dataf.ipynb` for code on assembling these sparse matrices. 